<a href="https://colab.research.google.com/github/jsebastianquiroga/PUJ_NLP_IA/blob/main/proyecto/motor_recomendaci%C3%B3n_texto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <img style="float: center; padding-right: 10px;" src="https://www.pikpng.com/pngl/b/467-4670553_universidad-javeriana-esslingen-am-neckar-clipart.png" width="100" height="150">



<h1> <strong>Procesamiento de Lenguaje Natural.</strong></h1>
<h2> Para: Ing. Juan Pablo Pajaro Hernandez, M.Sc.</br></h2>
<h2>Integrantes:</br></h2>
<h2>Oscar Eduardo Correcha Guzman.</br></h2>
<h2>Jhon Alberto Aguirre Ardila.</br></h2>
<h2>Juan Sebastián Quiroga Bernal. </h2>

**Este colab puede ser ejecutado por cualquier usuario sin necesidad de cargar los archivos al entorno, debido a que la conexión está configurada para acceder a los archivos desde la nube**.


<h1> <strong>Taller 1: Twitter User Gender Classification.</strong></h1>

En el presente Notebook, se exploraran los datos del data set de Kaggel: Twitter User Gender Classification. https://www.kaggle.com/crowdflower/twitter-user-gender-classification/

Este conjunto de datos, fue utilziado para entranar un modelo de clasificación de genero conocido como "CrowdFlower AI".

Cuyo proposito era inferir mendiante el perfil de un usuario de Twitter si el usuario era un hombre, mujer o compañia.

Se tienen un total de 20.000 observaciones, conformadas con un nombre de usuario, tuit aleatorio, perfil, la imagen de la cuenta, la ubicación.

<ol>
_unit_id: un id único para el usuario</ol>
<ol>_golden: si el usuario se incluyó en el patrón oro del modelo; TRUE o FALSE</ol>
<ol>_unit_state: estado de la observación; uno de finalized (para contributor-judged) o golden (para gold standard observations)</ol>
<ol>_trusted_judgments: número de juicios de confianza (int); siempre 3 para las observaciones no gold, y lo que puede ser un identificador único para las observaciones gold standard</ol>
<ol>_last_judgment_at: fecha y hora de la última resolución del contribuyente; en blanco para las observaciones gold standard</ol>
<ol>gender: uno de masculino, femenino o marca (para perfiles no humanos)</ol>
<ol>gender:confidence: valor flotante que representa la confianza en el sexo indicado</ol>
<ol>profile_yn: "no" aquí parece significar que el perfil debía formar parte del conjunto de datos pero no estaba disponible cuando los colaboradores fueron a juzgarlo</ol>
<ol>profile_yn:confidence: confianza en la existencia/no existencia del perfil</ol>
<ol>created: fecha y hora de creación del perfil</ol>
<ol>description: descripción del perfil del usuario</ol>
<ol>fav_number: número de tweets que el usuario ha marcado como favoritos</ol>
<ol>gender_gold: si el perfil es dorado, ¿cuál es el género?</ol>
<ol>link_color: el color del enlace en el perfil, como valor hexadecimal</ol>
<ol>name: nombre del usuario</ol>
<ol>profile_yn_gold: si el valor s/n del perfil es dorado</ol>
<ol>profileimage: un enlace a la imagen del perfil</ol>
<ol>retweet_count: número de veces que el usuario ha retuiteado (o posiblemente, ha sido retuiteado)</ol>
<ol>sidebar_color: color de la barra lateral del perfil, como valor hexadecimal</ol>
<ol>text: texto de un tweet aleatorio del usuario</ol>
<ol>tweet_coord: si el usuario tiene activada la localización, las coordenadas</ol>
<ol>como una cadena con el formato "[latitud, longitud]"</ol>
<ol>tweet_count: número de tweets que ha publicado el usuario</ol>
<ol>tweet_created: cuando se creó el tweet aleatorio (en la columna de texto)</ol>
<ol>tweet_id: el id del tweet aleatorio</ol>
<ol>tweet_location: ubicación del tweet; parece no estar especialmente normalizada</ol>
<ol>user_timezone: la zona horaria del usuario</ol>

El presente notebook, estará dividido en divido en las siguientes partes:
<ol>
<li> TBD:. </li>
<li> TBD. </li>
<li> TBD.</li>
</ol>

***

## <h1><strong> 1. Cargar el data set: </h1></strong>

El dataset del link en referencia, fue descargado previamente y subido a google drive, para poder cargar el presente notebook sin problemas al momento de cargar la información. </br>

In [ ]:
books.columns

Index(['book_id', 'goodreads_book_id', 'authors', 'original_publication_year',
       'isbn', 'original_title', 'title', 'language_code', 'average_rating',
       'summary'],
      dtype='object')

In [ ]:
# Clonar el repositorio de GitHub
!git clone https://github.com/zygmuntz/goodbooks-10k.git

# Importar las bibliotecas necesarias
import pandas as pd
import requests
from concurrent.futures import ThreadPoolExecutor

# Cargar las bases de datos
books = pd.read_csv('goodbooks-10k/books.csv')
books = books[['book_id', 'goodreads_book_id', 'authors', 'original_publication_year','isbn',
       'original_title', 'title', 'language_code', 'average_rating']]
tags = pd.read_csv('goodbooks-10k/tags.csv')
book_tags = pd.read_csv('goodbooks-10k/book_tags.csv')

# Definir la función para obtener el resumen del libro usando la API de Google Books
def get_google_book_summaries(isbn_list):
    # Construir la consulta con múltiples ISBNs
    query = "|".join([f"isbn:{isbn}" for isbn in isbn_list])
    url = f"https://www.googleapis.com/books/v1/volumes?q={query}"

    # Hacer la solicitud a la API
    response = requests.get(url)
    data = response.json()

    # Extraer los resúmenes de los libros si están disponibles
    summaries = {}
    if 'items' in data:
        for item in data['items']:
            isbn = item['volumeInfo']['industryIdentifiers'][0]['identifier']
            summary = item['volumeInfo'].get('description', None)
            summaries[isbn] = summary
    return summaries

# Dividir los ISBNs en lotes de 10 (o cualquier otro número que prefieras)
isbn_batches = [books['isbn'][i:i+10].tolist() for i in range(0, len(books), 10)]

# Paralelizar las solicitudes en lotes
with ThreadPoolExecutor(max_workers=5) as executor:
    summaries_list = list(executor.map(get_google_book_summaries, isbn_batches))

# Consolidar los resúmenes en un solo diccionario
all_summaries = {}
for summaries in summaries_list:
    all_summaries.update(summaries)

# Asignar los resúmenes al DataFrame
books['summary'] = books['isbn'].map(all_summaries)

# 1. Unir 'books' con 'book_tags' usando 'goodreads_book_id'
books_with_tags = pd.merge(books, book_tags, on='goodreads_book_id', how='left')

# 2. Unir el resultado con 'tags' usando 'tag_id'
books_with_tags_desc = pd.merge(books_with_tags, tags, on='tag_id', how='left')

# Explorar el DataFrame resultante
print(books_with_tags_desc.head())

fatal: destination path 'goodbooks-10k' already exists and is not an empty directory.
   book_id  goodreads_book_id          authors  original_publication_year  \
0        1            2767052  Suzanne Collins                     2008.0   
1        1            2767052  Suzanne Collins                     2008.0   
2        1            2767052  Suzanne Collins                     2008.0   
3        1            2767052  Suzanne Collins                     2008.0   
4        1            2767052  Suzanne Collins                     2008.0   

        isbn    original_title                                    title  \
0  439023483  The Hunger Games  The Hunger Games (The Hunger Games, #1)   
1  439023483  The Hunger Games  The Hunger Games (The Hunger Games, #1)   
2  439023483  The Hunger Games  The Hunger Games (The Hunger Games, #1)   
3  439023483  The Hunger Games  The Hunger Games (The Hunger Games, #1)   
4  439023483  The Hunger Games  The Hunger Games (The Hunger Games, #1)   



In [ ]:
books_with_tags_desc.summary.nunique()

0

In [ ]:
books_with_tags_desc.to_csv('books_with_description.csv')

## segundo enfoque.

In [2]:
# Importar las bibliotecas necesarias
# Clonar el repositorio de GitHub
!git clone https://github.com/zygmuntz/goodbooks-10k.git

import pandas as pd

# Cargar las bases de datos
books = pd.read_csv('goodbooks-10k/books.csv')
tags = pd.read_csv('goodbooks-10k/tags.csv')
book_tags = pd.read_csv('goodbooks-10k/book_tags.csv')

# Unir 'books' con 'book_tags' usando 'goodreads_book_id'
books_with_tags = pd.merge(books, book_tags, on='goodreads_book_id', how='left')

# Unir el resultado con 'tags' usando 'tag_id'
books_with_tags_desc = pd.merge(books_with_tags, tags, on='tag_id', how='left')


Cloning into 'goodbooks-10k'...
remote: Enumerating objects: 66, done.
remote: Total 66 (delta 0), reused 0 (delta 0), pack-reused 66
Receiving objects: 100% (66/66), 118.23 MiB | 31.49 MiB/s, done.
Resolving deltas: 100% (17/17), done.


In [3]:
# Ordenar los libros por su calificación promedio y número de calificaciones
top_books = books.sort_values(by=['average_rating', 'ratings_count'], ascending=False)

def recommend_books_based_on_popularity(n=5):
    return top_books.head(n)

In [4]:
def book_chatbot():
    print("¡Hola! ¿Te gustaría una recomendación de libro?")
    response = input("Responde sí o no: ").lower()

    if response == 'sí':
        n = int(input("¿Cuántos libros te gustaría que te recomiende? "))
        recommendations = recommend_books_based_on_popularity(n)
        print("\nAquí tienes tus recomendaciones:")
        for idx, row in recommendations.iterrows():
            print(f"- {row['title']} por {row['authors']} (Calificación: {row['average_rating']})")
    else:
        print("¡Está bien! Si cambias de opinión, ¡estoy aquí para ayudarte!")


In [6]:
book_chatbot()

¡Hola! ¿Te gustaría una recomendación de libro?
Responde sí o no: sí
¿Cuántos libros te gustaría que te recomiende? 5

Aquí tienes tus recomendaciones:
- The Complete Calvin and Hobbes por Bill Watterson (Calificación: 4.82)
- Words of Radiance (The Stormlight Archive, #2) por Brandon Sanderson (Calificación: 4.77)
- Harry Potter Boxed Set, Books 1-5 (Harry Potter, #1-5) por J.K. Rowling, Mary GrandPré (Calificación: 4.77)
- Mark of the Lion Trilogy por Francine Rivers (Calificación: 4.76)
- ESV Study Bible por Anonymous, Lane T. Dennis, Wayne A. Grudem (Calificación: 4.76)
